## SQL Databases

### Create sample SQLite Database

In [12]:
import sqlite3
import os

os.makedirs("data/databases",exist_ok=True)

conn = sqlite3.connect('data/databases/company.db')
cursor = conn.cursor()

In [6]:
# Create tables
cursor.execute('''CREATE TABLE IF NOT EXISTS employees
               (id INTEGER PRIMARY KEY, name TEXT, role TEXT, department TEXT, salary REAL)''')


cursor.execute('''CREATE TABLE IF NOT EXISTS projects
               (id INTEGER PRIMARY KEY, name TEXT, status TEXT, budget REAL, lead_id INTEGER)''')


### Insert sample data

In [7]:
employees = [
    (1, 'John Doe', 'Senior Developer', 'Engineering', 95000),
    (2, 'Jane Smith', 'Data Scientist', 'Analytics', 105000),
    (3, 'Mike Johnson', 'Product Manager', 'Product', 110000),
    (4, 'Sarah Williams', 'DevOps Engineer', 'Engineering', 98000)
]

projects = [
    (1, 'RAG Implementation', 'Active', 150000, 1),
    (2, 'Data Pipeline', 'Completed', 80000, 2),
    (3, 'Customer Portal', 'Planning', 200000, 3),
    (4, 'ML Platform', 'Active', 250000, 2)
]

In [8]:
cursor.executemany('INSERT OR REPLACE INTO employees VALUES (?,?,?,?,?)', employees)
cursor.executemany('INSERT OR REPLACE INTO projects VALUES (?,?,?,?,?)', projects)

In [13]:
res = cursor.execute("Select * from employees")

for r in res:
    print(r)

(1, 'John Doe', 'Senior Developer', 'Engineering', 95000.0)
(2, 'Jane Smith', 'Data Scientist', 'Analytics', 105000.0)
(3, 'Mike Johnson', 'Product Manager', 'Product', 110000.0)
(4, 'Sarah Williams', 'DevOps Engineer', 'Engineering', 98000.0)


In [14]:
conn.commit()
conn.close()


## Database Content Extraction

### 1. SQLDATABASE Utility

In [17]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

# Print tables
print(f"Tables: {db.get_usable_table_names()}")

Tables: ['employees', 'projects']


In [18]:
print(f"\nTable DDL:")
print(db.get_table_info())


Table DDL:

CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	John Doe	Senior Developer	Engineering	95000.0
2	Jane Smith	Data Scientist	Analytics	105000.0
3	Mike Johnson	Product Manager	Product	110000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	RAG Implementation	Active	150000.0	1
2	Data Pipeline	Completed	80000.0	2
3	Customer Portal	Planning	200000.0	3
*/


## 2. Custom SQL Processing

In [50]:
from typing import List
from langchain_core.documents import Document


def sql_to_documents(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    documents = []

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    # print(tables) # [('employees',), ('projects',)]

    for table in tables:
        table_name = table[0]
        
        # Get table schema
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        # print(columns) # [(0, 'id', 'INTEGER', 0, None, 1), (1, 'name', 'TEXT', 0, None, 0), (2, 'role', 'TEXT', 0, None, 0), (3, 'department', 'TEXT', 0, None, 0), (4, 'salary', 'REAL', 0, None, 0)]
        column_names = [col[1] for col in columns]
        # print(column_names) # ['id', 'name', 'role', 'department', 'salary']

        # Get table data
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        table_content = f"Table: {table_name}\n"
        table_content += f"Columns: {', '.join(column_names)}\n"

        table_content += "Sample records:\n"
        for row in rows:
            record = dict(zip(column_names, row))
            table_content += f"{record}\n"

        doc = Document(
            page_content=table_content,
            metadata={
                'source': db_path,
                'table_name': table_name,
                'num_records': len(rows),
                'data_type': 'sql_table'
            }
        )
        documents.append(doc)
    conn.close()
    return documents

In [51]:
sql_to_documents("data/databases/company.db")

[Document(metadata={'source': 'data/databases/company.db', 'table_name': 'employees', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Table: employees\nColumns: id, name, role, department, salary\nSample records:\n{'id': 1, 'name': 'John Doe', 'role': 'Senior Developer', 'department': 'Engineering', 'salary': 95000.0}\n{'id': 2, 'name': 'Jane Smith', 'role': 'Data Scientist', 'department': 'Analytics', 'salary': 105000.0}\n{'id': 3, 'name': 'Mike Johnson', 'role': 'Product Manager', 'department': 'Product', 'salary': 110000.0}\n{'id': 4, 'name': 'Sarah Williams', 'role': 'DevOps Engineer', 'department': 'Engineering', 'salary': 98000.0}\n"),
 Document(metadata={'source': 'data/databases/company.db', 'table_name': 'projects', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Table: projects\nColumns: id, name, status, budget, lead_id\nSample records:\n{'id': 1, 'name': 'RAG Implementation', 'status': 'Active', 'budget': 150000.0, 'lead_id': 1}\n{'id': 2, 'name': 'Dat

### Create relationship documents


In [ ]:
documents=[]
db_path = "data/databases/company.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
    SELECT e.name, e.role, p.name as project_name, p.status
    FROM employees e 
    JOIN projects p ON e.id = p.lead_id
""")

relationships = cursor.fetchall()

rel_content = "Employee-Project Relationships:\n\n"
for rel in relationships:
    # print(rel) # ('John Doe', 'Senior Developer', 'RAG Implementation', 'Active')
      += f"{rel[0]} ({rel[1]}) leads {rel[2]} - Status: {rel[3]}\n"
    # print(rel_content) # John Doe (Senior Developer) leads RAG Implementation - Status: Active

rel_doc = Document(
    page_content=rel_content,
    metadata={
        'source': db_path,
        'data_type': 'sql_relationships',
        'query': 'employee_project_join'
    }
)
documents.append(rel_doc)
conn.close()

print(documents)

[Document(metadata={'source': 'data/databases/company.db', 'data_type': 'sql_relationships', 'query': 'employee_project_join'}, page_content='Employee-Project Relationships:\n\nJohn Doe (Senior Developer) leads RAG Implementation - Status: Active\nJane Smith (Data Scientist) leads Data Pipeline - Status: Completed\nMike Johnson (Product Manager) leads Customer Portal - Status: Planning\nJane Smith (Data Scientist) leads ML Platform - Status: Active\n')]
